# Library

In [331]:
import pandas as pd
import numpy as np
import os
import re
import collections
import unidecode
import nltk
from nltk.corpus import stopwords
import itertools 
from nltk.tokenize import word_tokenize
from string import punctuation
from functools import reduce
import ast
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/egarcia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [332]:
pd.set_option('display.max_colwidth', 100)

In [333]:
%matplotlib inline
from matplotlib import pyplot as plt

# Functions

In [334]:
def read_texts(path):
    data = []
    file_name = os.listdir(path)

    for name in file_name:
        if name.endswith('.txt'):
            with open(path + name,encoding="utf8") as f:
                text = f.read()
                data.append({'nombre':name.replace('.txt',''), 'texto':text})

    df = pd.DataFrame(data)
    return df

In [335]:
def clean_text(string):
    """
    A method to clean text 
    """
    
    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuation:
            if x != '/':
                string = string.replace(x, "")
            else:
                string = string.replace(x, " ")
    
    string = unidecode.unidecode(string)

#     # Converting the text to lower
#     string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in swords])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string 

In [336]:
nltk.download('stopwords')
swords = list(set(stopwords.words('spanish')))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/egarcia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [337]:
def sep_num_text(data):

    words = word_tokenize(data) 
    for w in words:
        if re.search(r'\d', w):
            ind = words.index(w)
            words[ind] = [''.join(g) for k, g in itertools.groupby(w, str.isalpha)]
    data = ' '.join([x if type(x) is not list else ' '.join(x) for x in words])
    
    return data

In [338]:
def get_f_b_context_text(data):
    
    '''Return context in a string format'''
    
    all_data = []
    sf_not_found = []
    
    for instance in data:
           
        texto = instance['texto']    
        #target_word = instance['short_form']
        target_word = instance['Abbreviation']
        
        if target_word in texto:

            stop_ini_idx = instance['StartOffset'] #índice del inicio de la target
            stop_fin_idx = instance['EndOffset'] #índice del inicio de la target

            _instance = []
            xf = texto[:stop_ini_idx] + ' <start> ' +texto[stop_ini_idx:stop_fin_idx] + ' <end> ' #palabras anteriores a la target
            #xf = texto[:stop_ini_idx] +texto[stop_ini_idx:stop_fin_idx]
            xb = texto[stop_fin_idx+1:]   #palabras posteriores a la target            

            instance_id = instance['Definition'] #id del significado
            #instance_id = instance['long_form']

            _instance.append(target_word)
            _instance.append(xf)
            _instance.append(xb)
            _instance.append(instance_id)

            all_data.append(_instance[:])
        else:
            sf_not_found.append(target_word)
#             print("El acrónimo {} no aparece en el texto {}".format(target_word, instance['doc_id']))
        
    return all_data, sf_not_found

In [339]:
def limit_context(data):

    for doc in data:
        
        sf, xf, xb, lf = doc[0], doc[1], doc[2], doc[3]

        xf_words = word_tokenize(xf)[-n_step_f-1:]
        xb_words = word_tokenize(xb)[:n_step_b]

        doc[1] = ' '.join(xf_words)
        doc[2] = ' '.join(xb_words)   

    return data        

In [340]:
def create_dict(data):
    
    data_dic = []
    
    for instance in data:
            
        dic = {}

        dic['short_form'] = instance[0]
        dic['context'] = instance[1] + ' ' + instance[2]
        dic['long_form'] = instance[3]
    
        data_dic.append(dic)
        
    return data_dic
    

In [341]:
def distance_levenshtein(str1, str2):
    d=dict()
    for i in range(len(str1)+1):
        d[i]=dict()
        d[i][0]=i
    for i in range(len(str2)+1):
        d[0][i] = i
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            d[i][j] = min(d[i][j-1]+1, d[i-1][j]+1, d[i-1][j-1]+(not str1[i-1] == str2[j-1]))
    return d[len(str1)][len(str2)]

In [342]:
def normalize_lf(row):
    leven2 = []
    for i in row:
        for j in row:
            if i != j:
                long = max(len(i),len(j))
                ratio = distance_levenshtein(i,j)/long
                if ratio < 0.2:
                    leven2.append(j)
    if leven2:
        leven2 = set(leven2)
        lista = []
        for i in leven2:
            #val = frec[frec['index'] == i]['long_form'].iloc[0]
            val = frec[frec['index'] == i]['Definition'].iloc[0]
            lista.append((i, val))
        lista = set(lista)
        most_freq = sorted(set(lista), key=lambda x: x[1], reverse = True)[0][0]
        sust = {}
        for i in set(leven2):
            sust[i] = most_freq
        
        return sust
    else:
        pass
    

In [343]:
def get_label(row):
    if row['long_form_x'] == row['long_form_y']:
        return 1
    else:
        return 0

In [344]:
def offsetA(row):
    return row['texto'].find(row['Mention_A'])
    
def offsetB(row):
    return row['texto'].find(row['Mention_B'])

def offsetB_end(row):
    return row['texto'].find(row['Mention_B']) + len(row['Mention_B'])

def offsetA_end(row):
    return row['Mention_A_StartOffset'] + len(row['Mention_A'])

In [345]:
def offset(row):
    return row['texto'].find(row['abrev'])

def offsetend(row):
    return row['StartOffset']+len(row['abrev'])

In [346]:
def defin_dictionary(row,dictionary):
    if row['Definition'] == 'no_existe':
        return dictionary.get(row['Abbreviation'])
    else:
        return row['Definition']

# Load Data

### Testing model output

220 clinical cases.

No haría falta procesarlo pues crearemos el fichero directamente d elas notas clínicas de test. Luego lo pasaremos por el transformer y la salida la procesaremos para que sea como el gold standard. Aplicaremos el evaluador de IberEval.

In [347]:
test_out = pd.read_csv("../../data/julio23/test_data_beto_10_allacronim_normalizedlf_abremesprocessed_julio23_OUTPUT.csv")

In [348]:
test_out.head()

,Unnamed: 0,short_form,context,long_form,sentences,Prediction
0,0,mm,año se objetivó un pequeño angioma protuberancial izquierdo de 3 < start > mm < end > de diámetr...,milímetro,[CLS] milímetro [SEP] año se objetivó un pequeño angioma protuberancial izquierdo de 3 < start >...,0.999042
1,1,TC,4/5 y una hemihipoestesia derecha con extinción sensitiva . La < start > TC < end > craneal real...,tomografaa computerizada,[CLS] tomografaa computerizada [SEP] 4/5 y una hemihipoestesia derecha con extinción sensitiva ....,0.998629
2,2,TC,4/5 y una hemihipoestesia derecha con extinción sensitiva . La < start > TC < end > craneal real...,trayectorias clanicas,[CLS] trayectorias clanicas [SEP] 4/5 y una hemihipoestesia derecha con extinción sensitiva . La...,0.969132
3,3,TC,4/5 y una hemihipoestesia derecha con extinción sensitiva . La < start > TC < end > craneal real...,trastornos cra3nicos,[CLS] trastornos cra3nicos [SEP] 4/5 y una hemihipoestesia derecha con extinción sensitiva . La ...,0.991494
4,4,TC,4/5 y una hemihipoestesia derecha con extinción sensitiva . La < start > TC < end > craneal real...,taninos condensados,[CLS] taninos condensados [SEP] 4/5 y una hemihipoestesia derecha con extinción sensitiva . La <...,0.994472


In [349]:
test_out.Prediction.unique()

array([0.99904233, 0.99862933, 0.969132  , ..., 0.9663539 , 0.79747665,
       0.99821174])

In [350]:
test_out.Prediction.min()

0.0044755517

In [351]:
test_out.Prediction.describe()

count    9009.000000
mean        0.916268
std         0.198598
min         0.004476
25%         0.963787
50%         0.994676
75%         0.998046
max         0.999572
Name: Prediction, dtype: float64

In [352]:
len(test_out)

9009

## Join doc_ids by dataframe index

In [353]:
df_ids = pd.read_csv('../../data/julio23/test_data_beto_10_allacronim_normalizedlf_abremesprocessed_julio23_IDS.csv')

In [354]:
len(df_ids)

9009

In [355]:
df_test = test_out.join(df_ids)

In [356]:
df_test = df_test.rename(columns = {'doc_id':'# Document_ID', 'short_form':'Abbreviation'})

In [357]:
df_test.head()

,Unnamed: 0,Abbreviation,context,long_form,sentences,Prediction,# Document_ID
0,0,mm,año se objetivó un pequeño angioma protuberancial izquierdo de 3 < start > mm < end > de diámetr...,milímetro,[CLS] milímetro [SEP] año se objetivó un pequeño angioma protuberancial izquierdo de 3 < start >...,0.999042,S1130-14732005000200003-1
1,1,TC,4/5 y una hemihipoestesia derecha con extinción sensitiva . La < start > TC < end > craneal real...,tomografaa computerizada,[CLS] tomografaa computerizada [SEP] 4/5 y una hemihipoestesia derecha con extinción sensitiva ....,0.998629,S1130-14732005000200003-1
2,2,TC,4/5 y una hemihipoestesia derecha con extinción sensitiva . La < start > TC < end > craneal real...,trayectorias clanicas,[CLS] trayectorias clanicas [SEP] 4/5 y una hemihipoestesia derecha con extinción sensitiva . La...,0.969132,S1130-14732005000200003-1
3,3,TC,4/5 y una hemihipoestesia derecha con extinción sensitiva . La < start > TC < end > craneal real...,trastornos cra3nicos,[CLS] trastornos cra3nicos [SEP] 4/5 y una hemihipoestesia derecha con extinción sensitiva . La ...,0.991494,S1130-14732005000200003-1
4,4,TC,4/5 y una hemihipoestesia derecha con extinción sensitiva . La < start > TC < end > craneal real...,taninos condensados,[CLS] taninos condensados [SEP] 4/5 y una hemihipoestesia derecha con extinción sensitiva . La <...,0.994472,S1130-14732005000200003-1


In [358]:
df_test.shape

(9009, 7)

## Join Soto original df to add the SF offsets

In [359]:
test_soto = pd.read_csv("../../data/marzo2023/subtrack2/test_subtrack2_marzo23soto_parte1.csv", sep = '\t')

In [360]:
test_soto = test_soto[['# Document_ID', 'StartOffset', 'EndOffset', 'Abbreviation']]

In [361]:
test_soto = test_soto.drop_duplicates()

In [362]:
#test_soto.sort_values(by = ['# Document_ID','Abbreviation'],ascending = False).head()
test_soto.sort_values(by = '# Document_ID', ascending = False).head(10)

,# Document_ID,StartOffset,EndOffset,Abbreviation
9156,S1889-836X2015000200005-2,433,436,IMC
9239,S1889-836X2015000200005-2,1913,1916,MTT
9153,S1889-836X2015000200005-2,141,142,D
9207,S1889-836X2015000200005-2,812,814,Rx
9208,S1889-836X2015000200005-2,878,881,RMN
9216,S1889-836X2015000200005-2,943,946,MTT
9221,S1889-836X2015000200005-2,1061,1064,PTH
9229,S1889-836X2015000200005-2,1076,1077,D
9232,S1889-836X2015000200005-2,1297,1298,D
9238,S1889-836X2015000200005-2,1867,1869,Rx


In [363]:
test_soto.shape

(1561, 4)

In [364]:
#test_soto[(test_soto['# Document_ID'] == 'S1130-14732005000200003-1') & (test_soto['Abbreviation'] == 'mm')]

In [365]:
#df_test[(df_test['# Document_ID'] == 'S1130-14732005000200003-1') & (df_test['Abbreviation'] == 'mm')]

In [366]:
df_test2 = df_test.merge(test_soto, on = ['# Document_ID','Abbreviation'])

In [367]:
df_test2.sort_values(by = '# Document_ID', ascending = False).head()

,Unnamed: 0,Abbreviation,context,long_form,sentences,Prediction,# Document_ID,StartOffset,EndOffset
4207,2146,MTT,una Rx que mostró callo de fractura antiguo en 2o < start > MTT < end > por fractura de estrés p...,modelo transtea3rico,[CLS] modelo transtea3rico [SEP] una Rx que mostró callo de fractura antiguo en 2o < start > MTT...,0.982404,S1889-836X2015000200005-2,1913,1916
4204,2142,MTT,", que reveló una fractura de estrés en el 2o < start > MTT < end > con callo perióstico y edema ...",metil tiazol tetrazolium,"[CLS] metil tiazol tetrazolium [SEP] , que reveló una fractura de estrés en el 2o < start > MTT ...",0.997818,S1889-836X2015000200005-2,1913,1916
4220,2145,PTH,. Se realizó estudio analítico destacando incremento de niveles de < start > PTH < end > y vitam...,parathyroid hormone,[CLS] parathyroid hormone [SEP] . Se realizó estudio analítico destacando incremento de niveles ...,0.931628,S1889-836X2015000200005-2,1061,1064
4219,2144,PTH,. Se realizó estudio analítico destacando incremento de niveles de < start > PTH < end > y vitam...,parathormona intacta,[CLS] parathormona intacta [SEP] . Se realizó estudio analítico destacando incremento de niveles...,0.984401,S1889-836X2015000200005-2,1061,1064
4217,2151,MTT,para completar estudio que mostró edema de 1er y 3er < start > MTT < end > de huesos cuneiforme ...,metil tiazol tetrazolium,[CLS] metil tiazol tetrazolium [SEP] para completar estudio que mostró edema de 1er y 3er < star...,0.982785,S1889-836X2015000200005-2,2054,2057


In [368]:
len(df_test2)

19227

### Select just the predictions higher than 0.75 as the correct LF

In [369]:
df_test2 = df_test2[df_test2.Prediction >= 0.75] 

In [370]:
len(df_test2)

17428

In [371]:
df_test2.Prediction.unique()

array([0.99904233, 0.9988949 , 0.9990246 , ..., 0.9663539 , 0.79747665,
       0.99821174])

In [372]:
df_test2 = df_test2[['# Document_ID', 'StartOffset', 'EndOffset','Abbreviation','long_form']]

In [373]:
df_test2 = df_test2.rename(columns = {'long_form':'Definition'})

### Get lemmatized long forms

Do that after the output from Transformer

In [374]:
lemmatizer = WordNetLemmatizer()

In [375]:
df_test2['Definition_lemmatized'] = df_test2['Definition'].map(lambda x: lemmatizer.lemmatize(x))

In [376]:
df_test2.head()

,# Document_ID,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S1130-14732005000200003-1,300,302,mm,milímetro,milímetro
1,S1130-14732005000200003-1,741,743,mm,milímetro,milímetro
2,S1130-14732005000200003-1,1980,1982,mm,milímetro,milímetro
3,S1130-14732005000200003-1,300,302,mm,milímetro,milímetro
4,S1130-14732005000200003-1,741,743,mm,milímetro,milímetro


In [377]:
df_test2.shape

(17428, 6)

In [378]:
df_test2 = df_test2.drop_duplicates()

In [379]:
df_test2.shape

(8626, 6)

#### Write

In [380]:
df_test2.to_csv("../../data/julio23/test_data_beto_10_NOamb_normalizedlf_medline_julio23_output_processed.tsv", sep = '\t', index = False)

## Compare gold standard and test

In [206]:
gold = pd.read_csv("../../data/marzo2023/subtrack2/clinical_cases.abbreviations.testing_set.tsv", sep = '\t')

In [207]:
gold.head()

,# Document_ID,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0212-71992005000500009-1,533,537,/mm3,milímetro cúbico,milímetro cúbico
1,S1130-14732005000200003-1,1857,1858,µ,micro,micro
2,S0211-69952013000200018-1,2512,2514,µg,microgramo,microgramo
3,S0212-16112004000400007-1,4558,4560,µg,microgramo,microgramo
4,S1137-66272014000300016-1,1112,1114,µg,microgramo,microgramo


In [205]:
df_test2.head()

,# Document_ID,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S1130-14732005000200003-1,300,302,mm,aminoacidos más abundantes,aminoacidos más abundantes
1,S1130-14732005000200003-1,741,743,mm,aminoacidos más abundantes,aminoacidos más abundantes
2,S1130-14732005000200003-1,1980,1982,mm,aminoacidos más abundantes,aminoacidos más abundantes
9,S1130-14732005000200003-1,649,651,TC,tomografía computerizada,tomografía computerizada
10,S1130-14732005000200003-1,649,651,TC,trayectorias clínicas,trayectorias clínicas


In [210]:
gold[gold['# Document_ID'] == 'S0212-71992005000500009-1'].sort_values(by = 'StartOffset')

,# Document_ID,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0212-71992005000500009-1,533,537,/mm3,milímetro cúbico,milímetro cúbico
1146,S0212-71992005000500009-1,598,600,gr,gramo,gramo
1611,S0212-71992005000500009-1,601,602,L,litro,litro
1686,S0212-71992005000500009-1,638,641,LDH,lactato-deshidrogenasa,lactato-deshidrogenasa
3200,S0212-71992005000500009-1,646,647,U,unidad,unidad
1610,S0212-71992005000500009-1,648,649,L,litro,litro
2443,S0212-71992005000500009-1,698,702,mmol,milimol,milimol
1608,S0212-71992005000500009-1,703,704,L,litro,litro
48,S0212-71992005000500009-1,706,709,ADA,adenosín de aminasa,adenosín de aminasa
3199,S0212-71992005000500009-1,713,714,U,unidad,unidad


In [212]:
test_soto[test_soto['# Document_ID'] == 'S0212-71992005000500009-1'].sort_values(by = 'StartOffset')

,# Document_ID,StartOffset,EndOffset,Abbreviation
39746,S0212-71992005000500009-1,452,453,L
39733,S0212-71992005000500009-1,638,641,LDH
39745,S0212-71992005000500009-1,646,647,U
39812,S0212-71992005000500009-1,706,709,ADA
39833,S0212-71992005000500009-1,1050,1052,TC
39949,S0212-71992005000500009-1,1233,1235,TC
40065,S0212-71992005000500009-1,1549,1552,PCR


In [214]:
df_test2[(df_test2['# Document_ID'] == 'S0212-71992005000500009-1') & (df_test2.Abbreviation == 'LDH')].sort_values(by = 'StartOffset')

,# Document_ID,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
41542,S0212-71992005000500009-1,638,641,LDH,Lactato Deshidrogenasa,Lactato Deshidrogenasa
41543,S0212-71992005000500009-1,638,641,LDH,LA DESHIDROGENASA L'ACTICA,LA DESHIDROGENASA L'ACTICA


In [216]:
df_test2[(df_test2['# Document_ID'] == 'S0212-71992005000500009-1') & (df_test2.Abbreviation == 'PCR')].sort_values(by = 'StartOffset')

,# Document_ID,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
41885,S0212-71992005000500009-1,1549,1552,PCR,especificidad del 100 por ciento,especificidad del 100 por ciento
41886,S0212-71992005000500009-1,1549,1552,PCR,Amplicor®-MT Roche Diagnostic,Amplicor®-MT Roche Diagnostic
41888,S0212-71992005000500009-1,1549,1552,PCR,primeramente un diagnóstico molecular,primeramente un diagnóstico molecular
41889,S0212-71992005000500009-1,1549,1552,PCR,programa de cirugía robótica,programa de cirugía robótica
41890,S0212-71992005000500009-1,1549,1552,PCR,Parotiditis crónica recidivante en niños,Parotiditis crónica recidivante en niños
41891,S0212-71992005000500009-1,1549,1552,PCR,hepatitis C y DNA viral,hepatitis C y DNA viral
41892,S0212-71992005000500009-1,1549,1552,PCR,Programa de Comunicación de Riesgos,Programa de Comunicación de Riesgos
41893,S0212-71992005000500009-1,1549,1552,PCR,PROTEÍNA C REACTIVA,PROTEÍNA C REACTIVA
41896,S0212-71992005000500009-1,1549,1552,PCR,macroangiopatía y citocinas proinflamatorias,macroangiopatía y citocinas proinflamatorias
41897,S0212-71992005000500009-1,1549,1552,PCR,punto de corte 1mg/dl,punto de corte 1mg/dl


In [217]:
test_soto.Abbreviation.value_counts()

cm     144
TAC     85
mg      63
TC      58
mm      42
      ... 
TGO      1
ETE      1
MEN      1
ENA      1
ERG      1
Name: Abbreviation, Length: 319, dtype: int64